### Import

In [27]:
import numpy as np
from numpy import array, log, exp, where, vectorize
from bond_pricing import annuity_pv, equiv_rate, annuity_fv, annuity_rate, annuity_instalment

### Converting Between Rate Periodicity 

Converting between discrete periodicities:
$$\Big(1+\frac{APR_{m}}{m}\Big)^{m} = \Big(1+\frac{APR_{n}}{n}\Big)^{n}$$
$$n\times\Big[\Big(1+\frac{APR_{m}}{m}\Big)^{m/n}-1\Big] = APR_{n}$$

Converting to and from continuous compounding:
$$FV = PV \times e^{rt}$$
$$R_{m} = m\times(e^{R_{c}/m} - 1)$$
$$R_{c}=\ln\Big(1 + \frac{R_{c}}{m}\Big) \times m$$
- $R_c$ = continuously compounded interest rate.
- $R_m$ = periodically compounded interest rate, compounded m times per year
- $m$ = compounding times per year.
- $e$ = Euler's number, a constant with a value of roughly 2.71828.

In [3]:
rate = 0.04
from_freq = 4
to_freq = 1

In [4]:
print(equiv_rate(rate, from_freq=from_freq,to_freq=to_freq)*100)

4.060401000000002


In [5]:
def equivRate(rate, from_freq=1, to_freq=1):
  if from_freq == np.inf:
    return (exp(rate / to_freq) - 1) * to_freq
  elif to_freq == np.inf:
    return log(1 + rate / from_freq) * from_freq
  else:
    return to_freq * ((1 + rate/from_freq)**(from_freq/to_freq) - 1)

print(equivRate(rate, from_freq, to_freq)*100)

4.060401000000002


In [6]:
def equivRate(rate, from_freq=1, to_freq=1):
  if from_freq == np.inf:
    return (exp(rate / to_freq) - 1) * to_freq
  elif to_freq == np.inf:
    return log(1 + rate / from_freq) * from_freq
  else:
    rate = log(1 + rate / from_freq) * from_freq
    return (exp(rate / to_freq) - 1) * to_freq

print(equivRate(rate, from_freq, to_freq)*100)

4.060401000000002


In [7]:
def equivRate(rate, from_freq=1, to_freq=1):
  cc_rate = where(from_freq == np.inf, rate,
                  log((1 + rate / from_freq)) * from_freq)  
  res = where(from_freq == to_freq, rate,
                where(to_freq == np.inf, cc_rate,
                      to_freq * (exp(cc_rate / to_freq) - 1)))
  return res
  
print(equivRate(rate, from_freq, to_freq)*100)

4.060401000000002


### Annuity PV

Simple specification of a term structure: 
$$d_t = \Big(\frac{1}{1+r}\Big)^t=\frac{1}{(1+r)^t}$$

The current bond price $(PV_0)$ is the present value of the cash flows from the bond:
$$PV_0=\sum_{t=1}^{T}\frac{PMT}{(1+r)^n}+\frac{FV}{(1+r)^n}$$

Simplified bond price formula (ordinary):
$$PV_0=PMT\Bigg(\frac{1-(1+r)^{-n}}{r}\Bigg)+\frac{FV}{(1+r)^n}$$

In [8]:
rate = 0.07 / 2
n_period = 12 * 2
installment = 30 / 2
terminal_payment = 100
cf_freq = 1
comp_freq = 1
beg = False

In [9]:
print(annuity_pv(rate=rate, n_periods=n_period, instalment=installment, terminal_payment=terminal_payment, cf_freq=cf_freq, comp_freq=comp_freq, immediate_start=beg))

284.6712274346845


In [10]:
def pv(r, n, pmt, fv, beg=False):
  old_settings = np.seterr(invalid='ignore')
  pvPMT = where(r == 0, n, np.divide(1 - (1+r)**-n, r)) * pmt
  np.seterr(**old_settings)
  pvFV = fv / (1 + r)**n
  
  return where(beg, (pvPMT + pvFV) * (1 + r), pvPMT + pvFV)

print(pv(rate, n_period, installment, terminal_payment, beg)) 

284.6712274346845


In [11]:
def pv(r, n, pmt, fv, beg=False):
  c = np.full(n, pmt)
  t = np.arange(1, n+1)
  d = (1. / np.power((1 + r), t))
  B = np.sum(d * c)
  tv = fv / (1 + r)**n

  return where(beg, (B + tv) * (1 + r), B + tv)

print(pv(rate, n_period, installment, terminal_payment, beg)) 

284.6712274346851


### Annuity FV

Future Value ($FV_n$) of an Annuity formula (ordinary):
$$FV_{n} = PMT \times \frac{(1+r)^{n}-1}{r}+TV$$

Future Value ($FV_n$) of an Annuity formula (due):
$$FV_{n} = PMT \times \frac{(1+r)^{n}-1}{r} \times (1 + r)+TV$$

In [12]:
rate = 0.07 / 2
n_period = 7 * 2
installment = 100 / 2
terminal_payment = 1000
cf_freq = 1
comp_freq = 1
beg = True

In [13]:
print(annuity_fv(rate=rate, n_periods=n_period, instalment=installment, terminal_payment=terminal_payment, cf_freq=cf_freq, comp_freq=comp_freq,immediate_start=beg))

1949.7840439316562


In [14]:
def fv(r, n, pmt, terminal_payment, beg=False):
  old_settings = np.seterr(invalid='ignore')
  fvPMT = where(r == 0, n, np.divide((1+r)**n - 1, r)) * pmt
  np.seterr(**old_settings)
  TV = terminal_payment
  
  return where(beg, (fvPMT + TV) * (1 + r), fvPMT + TV)

print(fv(rate, n_period, installment, terminal_payment, beg)) 

1949.7840439316562


### Annuity Installments

Coupon payment formula can be derived by rearranging the following PV formula:
$$PV_0=PMT\Bigg(\frac{1-(1+r)^{-n}}{r}\Bigg)+\frac{FV}{(1+r)^n}$$

$$PMT=\Bigg[\frac{PV_0 - \frac{FV}{(1+r)^n}}{\frac{1-(1+r)^{-n}}{r}}\Bigg]$$

In [24]:
rate = 0.07 / 2
n_period = 12 * 2
pv0 = 284.6712274346845
terminal_payment = 100
cf_freq = 1
comp_freq = 1
beg = False

In [28]:
print(annuity_instalment(rate, n_period, pv0, terminal_payment=terminal_payment, immediate_start=beg))

15.0


In [30]:
def pmt(r, n, pv, fv, beg=False):
    return (pv - (fv/(1+r)**n)) / np.divide(1 - (1+r)**-n, r)

pmt(rate, n_period, pv0, terminal_payment, beg)

15.0

### Annuity Rate

https://en.wikipedia.org/wiki/Newton%27s_method

https://en.wikipedia.org/wiki/Secant_method



In [31]:
pv0 = 284.67
n_period = 12 * 2
installment = 30 / 2
terminal_payment = 100
cf_freq = 1
comp_freq = 1
beg = False

In [32]:
annuity_rate(n_periods=n_period, instalment=installment, pv=pv0, terminal_payment=terminal_payment, immediate_start=beg)

0.03500034621105279

In [34]:
from irr import irr_newton, irr_binary_search

arr = np.array(-pv0)
arr = np.append(arr, np.full(n_period, installment))
arr[n_period] += terminal_payment
print(arr)
print(irr_newton(arr))

[-284.67   15.     15.     15.     15.     15.     15.     15.     15.
   15.     15.     15.     15.     15.     15.     15.     15.     15.
   15.     15.     15.     15.     15.     15.    115.  ]
0.03500034621166748


In [35]:
from scipy.optimize import newton

def f(r): # The function whose zero is to be found
    return pv(r=r, n=n_period, pmt=installment, fv=terminal_payment, beg=False) - pv0
    
root, status = newton(f, 0, full_output=True, disp=False) 
print(root)

0.035000346211052774


In [33]:
arr = np.array(-pv0)
arr = np.append(arr, np.full(n_period, installment))
arr[n_period] += terminal_payment
print(irr_binary_search(arr))

0.035000346211667255


In [38]:
from bond_pricing import my_irr

def f(r): # The function whose zero is to be found
    return pv(r=r, n=n_period, pmt=installment, fv=terminal_payment, beg=False) - pv0
    
root = my_irr(f, 0) 
print(root)

0.03500034630298615


In [36]:
def approx_rate(n_period, installment, pv, fv, beg=False):
    return (installment + ((fv - pv) / n_period)) / ((fv + pv) / 2)

approx_rate(n_period, installment, pv0, terminal_payment, beg)

0.03798277311288463